In [ ]:
%%html
<style>
table {float:left}
</style>

## Data analysis:

### Train set

| Label name | | Number of images |
| - | | - | 
| hyundai | | 302 | 
| lexus | | 301 | 
| mazda | | 317 | 
| mercedes | | 342 | 
| opel | | 301 | 
| skoda | | 314 | 
| toyota | | 306 | 
| volkswagen | | 330 | 

### Test set
| Label name | | Number of images |
| - | | - | 
| hyundai | | 50 | 
| lexus | | 50 | 
| mazda | | 50 | 
| mercedes | | 50 | 
| opel | | 50 | 
| skoda | | 50 | 
| toyota | | 50 | 
| volkswagen | | 50 | 

In [ ]:
%matplotlib inline

### Imports

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize, ToPILImage
from torchvision.transforms.functional import to_grayscale, to_tensor, rotate
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image

from torch.utils.data import DataLoader

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from sklearn.metrics import classification_report, confusion_matrix

from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
import seaborn as sn
import numpy as np

### Useful functions to retrive the data

In [ ]:
# create a annotations_file.csv (index -> img (name), label (int))

def getLabelList(path):
    only_dirs = [ name for name in os.listdir(path) if 
                 os.path.isdir(os.path.join(path, name)) ]

    ret = {}
    index = 0
    
    for d in only_dirs:
        new_path = path + d
        label = only_dirs.index(d)
        # print(len(os.listdir(new_path)))
        for img in [ name for name in os.listdir(new_path) ]:
            ret[index] = [img, label]
            index += 1
    
    return pd.Series(ret)

In [ ]:
# create the pool of images (with a series that contains all the images, more deeply: index -> abs path, label (int), class (string))

def getImgPool(path):
    only_dirs = [ name for name in os.listdir(path) if 
                 os.path.isdir(os.path.join(path, name)) ]

    ret = {}
    index = 0
    
    for d in only_dirs:
        new_path = path + d
        label = only_dirs.index(d)
        # cnt = 0
        for img in [ name for name in os.listdir(new_path) ]:
            abs_path = new_path + '/' + img
            ret[index] = [abs_path, label, d]
            index += 1
            # cnt += 1
        # print(f"Label: {d} - Number: {cnt}")
    
    return pd.Series(ret)

### Defining the custom dataset (structure that holds the data)

In [ ]:
wrong_images_path = []
import traceback

class CustomImageDataset(Dataset):
    def __init__(self, path_labels, transform=None, target_transform=None):
        self.img_labels = getLabelList(path_labels)
        self.images = getImgPool(path_labels)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        # try: 
        image = read_image(self.images.loc[idx][0])
        # except Exception:
        #     print('Found error at {} - {}'.format(self.images.loc[img_name][0], self.img_labels.iloc[idx][1]))
        #     wrong_images_path.append(self.images.loc[img_name][0])
        #     return torch.zeros((600, 600, 1)), -1
        # image = read_image(self.images.loc[img_name][0])
        if image.shape[0] >= 3: #apply grayscale to colored img
            image = to_tensor(to_grayscale(ToPILImage()(image), num_output_channels=1))
        else: #we have a greyscaled image, move it into tensor 
            image = to_tensor(ToPILImage()(image))
            
        image = Resize((600, 600))(image)
        #data augmentation
        prob = torch.randint(100, size=(1,)).item()
        if prob > 20: 
            angle = torch.randint(60, size=(1,)).item() - 30
            image = rotate(image, angle)        
        
        image = image.permute(1,2,0) #made to be shown on matplotlib
        label = self.img_labels.loc[idx][1]
        
        return image, label

### Creating the 2 dataset and testing

In [ ]:
train_path = './data/car_brand_logos/Train/'
test_path = './data/car_brand_logos/Test/'

# train_data = CustomImageDataset(path_labels=train_path, transform=ToTensor(), num_of_channels=3)
# test_data = CustomImageDataset(path_labels=test_path, transform=ToTensor(), num_of_channels=3)

train_data = CustomImageDataset(path_labels=train_path, transform=ToTensor())

test_data = CustomImageDataset(path_labels=test_path, transform=ToTensor())

In [ ]:
# for i in range(train_data.__len__()):
#     img, label = train_data[i]

# for i in range(test_data.__len__()):
#     img, label = test_data[i]
    
    
# wrong_images_path

In [ ]:
#testing dataset if images are loaded

only_dirs = [ name for name in os.listdir(train_path) if 
                 os.path.isdir(os.path.join(train_path, name)) ]

diz = {}
nocl=0 #num of classes
for d in only_dirs:
    diz[nocl] = d
    nocl+=1

figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item() 
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(diz[label])
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap='gray')
    # plt.imshow(img.squeeze())
plt.show()

In [ ]:
# img, label = train_data[300]

# print(img.shape)

# plt.imshow(img.squeeze(), cmap='gray')

### creating dataloader (who iterate on data, create batches and shuffles it)


In [ ]:
train_dl = DataLoader(train_data, batch_size=8, shuffle=True)
test_dl = DataLoader(test_data, batch_size=8, shuffle=True)

#dataloader test (ask a batch or a sample)
images, labels = next(iter(train_dl)) #picking a batch
print(f"Feature batch shape: {images.size()}")
print(f"Labels batch shape: {labels.size()}")
img = images[0].squeeze() #picking the first img and label of the batch
label = labels[0]
plt.imshow(img, cmap='gray')
plt.show()
print(f"Label: {diz[label.item()]}")

In [ ]:
# os.rename('./data/car_brand_logos/Train/mercedes/r6.JPG', './data/car_brand_logos/Train/mercedes/r6.jpg')
# image = read_image('./data/car_brand_logos/Train/volkswagen/vosvos-volkswagen-logo-patches-arma-pec-kot-yamasi__1548797389090801.jpg')
# image = image.permute(1,2,0)
# plt.imshow(image, cmap='gray')
# plt.show()

### Defining the neural network

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

In [ ]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        #conv1: 1 input image channel (image channel, 1 gray, 3 rgb), 6 output channels (depth [K]), 5x5 square convolution kernel, DEFAULT: stride = 1,1, padding = 0
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 10, 5)        
        self.fc_layer1 = nn.Linear(10*147*147, 120)
        
        self.fc_layer2 = nn.Linear(120, 84)
        self.fc_layer3 = nn.Linear(84, nocl) # nocl is the num of classes

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square, you can specify with a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.fc_layer1(x))
        x = F.relu(self.fc_layer2(x))
        x = self.fc_layer3(x)
        return x

In [ ]:
net = Net()
# net.to(device)

In [ ]:
#test
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# use gradient descent
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Tensorboard setup

start tensorboard: `tensorboard --logdir=Users\eliad\Tesi\ProgettoTesi\runs\car_brands_1`

In [ ]:
writer = SummaryWriter('runs/car_brands_2')

In [ ]:
# get some random training images
dataiter = iter(train_dl)
images, labels = dataiter.next()

#visualizing and adding images into tb
for idx in range(4):
    plt.imshow(images[idx], cmap='gray')
    plt.show()
    img = images[idx]
    img = img.permute(2,0,1)
    writer.add_image(f"car_brands_images_{idx+1}", img)
    
writer.close()

### Adding the graph of the network into Tensorboard

In [ ]:
images = images.permute(0, 3, 1, 2) #move to tensor mode

writer.add_graph(net, images)
writer.close()

### Adding the projector 

In [ ]:
tmp_dl = DataLoader(train_data, batch_size=100, shuffle=True)
images, labels = next(iter(tmp_dl))

# get the class labels for each image
class_labels = [diz[lab.item()] for lab in labels]


images = images.permute(0,3,1,2) #tensor mode
images = Resize((28, 28))(images)

features = images.view(-1, 28 * 28)
writer.add_embedding(features,
                    metadata=class_labels,
                    label_img=images)
writer.close()

### Confusion matrix during training

In [ ]:
def createConfusionMatrix(loader):
    y_pred = [] # save predction
    y_true = [] # save ground truth

    # iterate over data
    for inputs, labels in loader:
        inputs = inputs.permute(0, 3, 1, 2)
        output = net(inputs)  # Feed Network

        output = (torch.max(torch.exp(output), 1)[1]).data.cpu().numpy()
        y_pred.extend(output)  # save prediction

        labels = labels.data.cpu().numpy()
        y_true.extend(labels)  # save ground truth

    # constant for classes
    classes = diz.values()

    # Build confusion matrix
    cf_matrix = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cf_matrix/np.sum(cf_matrix) * 10, index=[i for i in classes],
                         columns=[i for i in classes])
    plt.figure(figsize=(10, 7), dpi=150)    
    return sn.heatmap(df_cm, annot=True).get_figure()

### Training the network

In [ ]:
for epoch in range(3):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # inputs, labels = data[0].to(device), data[1].to(device)
        inputs = inputs.permute(0, 3, 1, 2) # to get a shape of [64, 1, 600, 600]

        # inputs = inputs.view(64, -1) #1° param specifico io, altri calcola lui (-1)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 45 == 44:    # print every 360 images (8 images per batch, 45 loops)
            print(f'[{epoch + 1}, {i+1}/{len(train_dl)}] loss: {running_loss:.3f}')

            # ...log the running loss
            writer.add_scalar('training loss',
                        running_loss / 1000,
                        epoch * len(train_dl) + i)
            running_loss = 0.0

    #end of an epoch, calculate loss on test set (since it's small and there's no need to split)
    test_loss = 0
    for _, data in enumerate(test_dl):
        inputs, labels = data
        inputs = inputs.permute(0, 3, 1, 2)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
    print(f"[{epoch + 1}] test loss: {test_loss:.3f}")
    # confusion matrix (added from extra step)
    writer.add_figure("Confusion matrix", createConfusionMatrix(test_dl), epoch)

print('Finished Training')

#saving the trained model
PATH = './trained/auto_brand2.pth'
torch.save(net.state_dict(), PATH)

### Testing the network on test set

In [ ]:
loaded = Net()
loaded.load_state_dict(torch.load("./trained/auto_brand.pth"))

In [ ]:
images, labels = next(iter(test_dl)) #picking a batch

for i in range(1):
    rnd_i = torch.randint(len(images), size=(1,)).item() 
    img = images[rnd_i].squeeze()
    label = labels[rnd_i]
    plt.imshow(img, cmap='gray')
    plt.suptitle('True: {}'.format(diz[label.item()]))
    
    #predicting the class of the image
    imgs = images.permute(0, 3, 1, 2)
    preds = net(imgs)
    _, best_pred = torch.max(preds.data, 1)
    plt.title('Predicted: {}'.format(diz[best_pred[rnd_i].item()]))
    # print(f"Numero di classi: {nocl}")
    # print(f"Dimensioni delle predizioni del batch da 64 elementi: {preds.size()}")
    # print(f"Predizioni dell'elemento scelto: {preds[rnd_i]}")
    # print(diz.values())
    
    plt.show()

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in diz}
total_pred = {classname: 0 for classname in diz}

y_true = []
y_pred = []

# again no gradients needed
with torch.no_grad():
    for data in test_dl:
        images, labels = data
        images = images.permute(0, 3, 1, 2)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[label.item()] += 1
            total_pred[label.item()] += 1
            y_true.append(label)
            y_pred.append(prediction)

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {diz[classname]} is {accuracy:.1f} %')

In [ ]:
#stampa report classificazioni
print(classification_report(y_true, y_pred, target_names=diz.values()))

In [ ]:
# writer.add_figure("Final matrix", createConfusionMatrix(test_dl), 0)